In [7]:
# !pip install elasticsearch
# !pip install csv2es
# !pip install requests
# !python -m pip install elasticsearch
from elasticsearch import helpers, Elasticsearch
import elasticsearch
import csv
import csv2es

In [8]:
es = Elasticsearch()

In [2]:
df = spark.read.csv('DNPBA2017.csv', header=True)

In [3]:
df_total = df.toPandas()

In [5]:
df_total.to_csv('DNPBA2017_total_es.csv', index=False)

In [22]:
df = df.select(['NUMERODN', 'CODESTAB', 'IDADEMAE', 'ESCMAEAGR1']).limit(20).toPandas()

In [23]:
df

,NUMERODN,CODESTAB,IDADEMAE,ESCMAEAGR1
0,72390242,2786095,25,01
1,72396320,NA,20,02
2,72374657,NA,20,04
3,72376065,2755157,28,06
4,72392109,7373120,34,12
5,72391035,2755157,23,12
6,72389213,2755157,30,04
7,75447854,2755157,16,04
8,69747564,2364816,17,02
9,69726416,2777770,33,08


In [24]:
df.to_csv('DNPBA2017_es.csv', index=False)

### indexng

In [6]:
with open('DNPBA2017_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='my-index', doc_type='my-type')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

NameError: name 'es' is not defined

In [11]:
with open('DNPBA2017_total_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='dnpba2017_total', doc_type='csv')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

### buscando termo

In [12]:
content = {
    "query": {
        "term": {
            "CODESTAB": "2786095"
         }
     },
}

In [13]:
res = es.search(index="my-index", body=content)

In [25]:
res['hits']['hits']

[{'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'HH5UancBCXt2NfhZ59dl',
  '_score': 2.6390574,
  '_source': {'NUMERODN': '72390242',
   'CODESTAB': '2786095',
   'IDADEMAE': '25',
   'ESCMAEAGR1': '01'}}]

### buscando um intervalo

In [29]:
content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }
}

In [30]:
res = es.search(index="my-index", body=content)

In [1]:
res['hits']

NameError: name 'res' is not defined

### busca lógica

In [75]:
content = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }
}

In [76]:
res = es.search(index="my-index", body=content)

In [77]:
res

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'LMatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 3.6686769,
    u'_source': {u'CODESTAB': u'2777770',
     u'ESCMAEAGR1': u'06',
     u'IDADEMAE': u'21',
     u'NUMERODN': u'69726563'},
    u'_type': u'my-type'}],
  u'max_score': 3.6686769,
  u'total': {u'relation': u'eq', u'value': 1}},
 u'timed_out': False,
 u'took': 5}

### searching methods

In [33]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'size': 30,
        'query': {
            'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [34]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'size': 100,
        'query': {
            'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [35]:
buscaExata("72390242", "2786095", "25", "01")

[{'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'HH5UancBCXt2NfhZ59dl',
  '_score': 10.55623,
  '_source': {'NUMERODN': '72390242',
   'CODESTAB': '2786095',
   'IDADEMAE': '25',
   'ESCMAEAGR1': '01'}}]

In [36]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

[]

In [37]:
buscaAproximada("72390242", "2786095", "25", "01")

[{'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'HH5UancBCXt2NfhZ59dl',
  '_score': 14.514815,
  '_source': {'NUMERODN': '72390242',
   'CODESTAB': '2786095',
   'IDADEMAE': '25',
   'ESCMAEAGR1': '01'}}]

In [38]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

[{'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'HH5UancBCXt2NfhZ59dl',
  '_score': 13.195287,
  '_source': {'NUMERODN': '72390242',
   'CODESTAB': '2786095',
   'IDADEMAE': '25',
   'ESCMAEAGR1': '01'}},
 {'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'JX5UancBCXt2NfhZ59dl',
  '_score': 1.0641158,
  '_source': {'NUMERODN': '69726416',
   'CODESTAB': '2777770',
   'IDADEMAE': '33',
   'ESCMAEAGR1': '08'}},
 {'_index': 'my-index',
  '_type': 'my-type',
  '_id': 'Jn5UancBCXt2NfhZ59dl',
  '_score': 1.0641158,
  '_source': {'NUMERODN': '69726448',
   'CODESTAB': '2777770',
   'IDADEMAE': '33',
   'ESCMAEAGR1': '06'}}]

In [39]:
print(len(buscaAproximada("72390242", "2786095", "33", "01")))

3
